In [1]:
%reload_ext autoreload
%autoreload
import pickle, os, json
import torch
from pathlib import Path, PureWindowsPath, PurePosixPath
from util.tools import load_config
import pandas as pd
from sklearn.metrics import classification_report

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [3]:
config_folder = "config/"
config = load_config(config_folder)
config.__dict__

{'hyperparams': <util.tools.config at 0x212a2f60460>,
 'programsettings': <util.tools.config at 0x212a2f70400>,
 'modelconfig': <util.tools.config at 0x212a2f70460>}

In [4]:
reports_folder_name = config.programsettings.REPORTS_DIR
data_folder = Path(reports_folder_name)
list_results_files = list(data_folder.glob('**/multi_model_*.pkl'))

all_results = []
all_data_source = []

for results_file in list_results_files:
#     print(results_file)
    with open(results_file,"rb") as f:
        lst_results = pickle.load(f)
    
    for results in lst_results:
        max_seq = 0
        lr = 0
        train_batch = 0
        max_seq = ""
        model_name = ""
        class_weights = '[]'
        kernel_1 = 0
        kernel_2 = 0
        kernel_3 = 0
        if (len(results[0]) > 50):
#             print(results[0])
            results_json = json.loads(results[0])
            max_seq = results_json["hyperparams"]["MAX_SEQ_LENGTH"]
            lr = results_json["hyperparams"]["LEARNING_RATE"]
            train_batch = results_json["hyperparams"]["TRAIN_BATCH_SIZE"]
            model_name = results_json["programsettings"]["MODEL_NAME"]
            try:
                class_weights = results_json["hyperparams"]["LOSS_FN_CLASS_WEIGHTS"]
            
            except: 
                class_weights = '[]'
                
            try:
                kernel_1 = results_json["modelconfig"]["KERNEL_1"]
                kernel_2 = results_json["modelconfig"]["KERNEL_2"]
                kernel_3 = results_json["modelconfig"]["KERNEL_3"]
            
            except: 
                class_weights = '[]'

        result_store = [results_file, model_name,lr, train_batch,max_seq, class_weights,kernel_1, kernel_2,kernel_3,results[1], results[2], results[3], results[4], results[5],results[6] ]
        all_results.append(result_store)
        
#       Creating list of all dev_labels and dev_preds, so that we can run classification reports for all of them together
        if (len(results) > 7):
            all_data_source.append([results[7],results[8], result_store])
                                
columns = ['file_name','model_name', 'lr', 'train_batch_size','train_max_seq','class_weights','Kernel_1','Kernel_2','Kernel_3','train_loss', 'dev_loss', 'train_mcc', 'train_f1_score','dev_mcc','dev_f1_score']

df_results = pd.DataFrame(all_results, columns = columns)
pd.set_option('display.max_colwidth', 100)
df_results

,file_name,model_name,lr,train_batch_size,train_max_seq,class_weights,Kernel_1,Kernel_2,Kernel_3,train_loss,dev_loss,train_mcc,train_f1_score,dev_mcc,dev_f1_score
0,reports\re\multi_model_experiment_results_2020-11-22 10_44_11_756583.pkl,BioBERT_fc,0.000010,12,128,[],0,0,0,0.008484,0.061662,0.995074,0.994248,0.981641,0.969833
1,reports\re\multi_model_experiment_results_2020-11-22 10_44_11_756583.pkl,BioBERT_fc,0.000008,12,128,[],0,0,0,0.008790,0.056426,0.994977,0.993931,0.981643,0.968364
2,reports\re\multi_model_experiment_results_2020-11-22 10_44_11_756583.pkl,BioBERT_fc,0.000013,12,128,[],0,0,0,0.008213,0.060076,0.995146,0.994219,0.982797,0.972130
3,reports\re\multi_model_experiment_results_2020-11-22 10_44_11_756583.pkl,BioBERT_fc,0.000013,12,256,[],0,0,0,0.008129,0.063987,0.995194,0.994399,0.981838,0.969729
4,reports\re\multi_model_experiment_results_2020-11-22 10_44_11_756583.pkl,BioBERT_fc,0.000013,24,256,[],0,0,0,0.008214,0.057145,0.995291,0.994401,0.982028,0.971121
5,reports\re\multi_model_experiment_results_2020-11-23 02_23_24_693151.pkl,BioBERT_fc,0.000010,24,256,[],0,0,0,0.008712,0.062036,0.995025,0.994146,0.982893,0.971911
6,reports\re\multi_model_experiment_results_2020-11-23 02_23_24_693151.pkl,BioBERT_fc,0.000010,24,256,[],0,0,0,0.014548,0.153791,0.994399,0.992004,0.981644,0.968190
7,reports\re\multi_model_experiment_results_2020-11-23 20_41_16_167499.pkl,BioBERT_CNN_fc,0.000010,24,256,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",8,10,12,1.396503,1.398815,0.964124,0.860661,0.960849,0.857321
8,reports\re\multi_model_experiment_results_2020-11-23 23_24_28_564691.pkl,BioBERT_CNN_fc,0.000010,24,256,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",12,14,16,1.395602,1.398685,0.965500,0.862644,0.961066,0.857194
9,reports\re\multi_model_experiment_results_2020-11-24 03_22_07_044108.pkl,BioBERT_CNN_fc,0.000010,24,256,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",24,30,36,1.395388,1.398834,0.965883,0.863039,0.960888,0.857717


### Let us generate Classification report for all the things we have available

In [5]:
for i in range(len(all_data_source)):
    dev_labels, dev_preds, model_config = all_data_source[i]
    print("\n \n \n",model_config)
    print(classification_report(dev_labels,dev_preds ))


 
 
 [WindowsPath('reports/re/multi_model_experiment_results_2020-11-22 10_44_11_756583.pkl'), 'BioBERT_fc', 1e-05, 12, '128', '[]', 0, 0, 0, 0.008483896204263553, 0.06166158813799957, 0.9950736886262432, 0.9942478125112407, 0.9816408026681803, 0.9698334137434981]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1034
           1       0.97      0.97      0.97      1145
           2       0.98      0.98      0.98      1382
           3       0.98      0.99      0.98      1269
           4       0.97      0.96      0.96       136
           5       0.96      0.96      0.96       954
           6       0.98      0.98      0.98       933
           7       0.93      0.90      0.91       250
           8       1.00      1.00      1.00      8201

    accuracy                           0.99     15304
   macro avg       0.97      0.97      0.97     15304
weighted avg       0.99      0.99      0.99     15304


 
 
 [WindowsPath('reports/r

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[WindowsPath('reports/re/multi_model_experiment_results_2020-11-23 23_24_28_564691.pkl'), 'BioBERT_CNN_fc', 1e-05, 24, 256, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 12, 14, 16, 1.3956016905421214, 1.3986851787866097, 0.9655000305435087, 0.8626441701783418, 0.9610663014716285, 0.8571944835550768]
              precision    recall  f1-score   support

           0       0.81      0.99      0.89      1034
           1       0.99      0.96      0.98      1145
           2       0.97      0.98      0.98      1382
           3       0.99      0.98      0.99      1269
           4       0.96      0.95      0.95       136
           5       0.96      0.97      0.97       954
           6       0.96      0.98      0.97       933
           7       0.00      0.00      0.00       250
           8       1.00      1.00      1.00      8201

    accuracy                           0.97     15304
   macro avg       0.85      0.87      0.86     15304
weighted avg       0.96      0.97      0.97    